In [1]:
%load_ext autoreload
%autoreload 2

#standard imports
import numpy as np
import pandas as pd
import os
from datetime import timedelta

# bokeh
from bokeh.io import output_notebook, export_png, export_svgs
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file, ColumnDataSource
output_notebook()

# lib
import sys
sys.path.append('../')
from metrics import compute_metrics, _compute_metrics
from analysis import (
    load_backfill, 
    plot_cases, 
    plot_metric_for_dates, 
    plot_error, 
    plot_prediction_interval,
    plot_accuracy,
    show,
)

Loading BokehJS ...

In [2]:
case_type = ('cases', 'infections')
region = 'Italy'
region_short = 'it'
f_ground_truth = f'../data/italy/data-upc.csv'

other_forecasts = {
    #'YYG': (f'/checkpoint/mattle/covid19/csvs/{case_type[1]}/yyg/counts_{{}}.csv', '#f29111', 'solid'),
    'Los Alamos': (f'/checkpoint/mattle/covid19/csvs/{case_type[1]}/los_alamos/counts_{{}}.csv', "#009ed7", 'solid')
}

### Progression of Cases

In [5]:
# Load ground truth data
df_region = pd.read_csv(f_ground_truth, index_col='region').transpose()
df_region.index.set_names(['date'], inplace=True)
df_region.index = pd.to_datetime(df_region.index)
print('Days = {}, Regions = {}'.format(*df_region.shape))
display(df_region.tail())

# plot cases over time 
print(df_region.shape)
p = plot_cases(df_region, f"Confirmed cases in {region}", show_hover=False)
show(p, f'img/{region_short}-cases.png')

Days = 96, Regions = 106


region,Agrigento,Alessandria,Ancona,Arezzo,Ascoli Piceno,Asti,Avellino,Bari,Barletta-Andria-Trani,Belluno,...,Udine,Valle d’Aosta,Varese,Venezia,Verbano-Cusio-Ossola,Vercelli,Verona,Vibo Valentia,Vicenza,Viterbo
date,,,,,,,,,,,,,,,,,,,,,
2020-05-29,141,3903,1870,676,290,1811,543,1474,383,1166,...,982,1182,3590,2660,1109,1295,5093,81,2845,429
2020-05-30,141,3922,1871,676,290,1818,546,1481,383,1167,...,982,1183,3594,2661,1109,1300,5098,81,2845,430
2020-05-31,141,3925,1872,676,290,1831,547,1483,383,1170,...,983,1184,3619,2661,1109,1304,5099,81,2846,430
2020-06-01,141,3927,1872,677,290,1835,547,1486,383,1171,...,984,1187,3622,2661,1109,1304,5099,81,2846,430
2020-06-02,141,3944,1872,677,290,1840,547,1487,383,1171,...,986,1187,3632,2665,1111,1305,5100,81,2849,430


(96, 106)


### Load Backfill and Configs

In [6]:
job = "it/2020_07_04_15_01"
job = os.environ.get('EXPERIMENT_ID', job)
model = os.environ.get('EXPERIMENT_MOD', 'car')
metric = "mae"

fs, cfgs = load_backfill(job, model=model, forecast=f"best_{metric}")
cfgs.drop(columns=['fdat', 'fpop', 'job'])

,activation,decay,eta,granger,loss,lr,momentum,niters,t0,temporal,test_on,weight_decay,window
date,,,,,,,,,,,,,
2020-04-22,sigmoid,latent2_2,0.3,0.5,nb,0.001,0.99,30000,5,0.5,14,0.1,25
2020-04-29,sigmoid,latent2_2,0.2,0.5,nb,0.001,0.99,30000,5,2.0,14,0.2,20
2020-05-05,sigmoid,latent2_2,0.1,0.5,nb,0.001,0.99,30000,5,2.0,14,0.2,20
2020-04-19,sigmoid,latent2_2,0.3,0.5,nb,0.001,0.99,30000,5,0.5,14,0.1,25
2020-05-01,sigmoid,latent2_2,0.1,0.5,nb,0.001,0.99,30000,5,1.0,14,0.1,20


### Forcast Comparison

Compare our forecasts to published data by 
- [Los Alamos National Laboratory](https://covid-19.bsvgateway.org/)
- [YYG](https://covid19-projections.com)

In [8]:
ps = plot_metric_for_dates(fs, df_region, cfgs.index, 'MAE')
grid = gridplot(ps, ncols=2, plot_width=430)
show(grid, f'img/{region_short}_{metric}.png')

In [9]:
df_ar = pd.read_csv(fs['2020-04-22'], index_col='date', parse_dates=['date'])
p = plot_error(df_ar, df_region, f'Prediction Error - {region}')
show(p, f'img/{region_short}_prediction_error.png')

In [10]:
ps = []
accs = []
plevel = (.01, .99)

def select_piv(df, interval):
    _df = df.drop(columns=['piv'])
    return _df[df["piv"] == interval].set_index('date')

for date in cfgs.index:
    jobdir = cfgs.loc[date]['job']
    fname = f'{jobdir}/../forecasts/piv_best_{metric}.csv'
    if not os.path.exists(fname):
        continue
    df_piv = pd.read_csv(fname, parse_dates=['date'])
    lower = select_piv(df_piv, str(plevel[0]))
    upper = select_piv(df_piv, str(plevel[1]))
    mean = select_piv(df_piv, "mean")

    ix = np.intersect1d(mean.index, df_region.index)
    df_gt = df_region.loc[ix]
    mean = mean.loc[ix]
    lower = lower.loc[ix]
    upper = upper.loc[ix]                         

    p = figure(title=f"Prediction Intervals {date[-5:]}, {len(mean)} days", 
               plot_width=300, plot_height=300, tools='save,hover',
              x_axis_label="Forecast days", y_axis_label="Cases",
              tooltips=[("Name", "$name"), ("Value", "$y")])
    regions = df_region.columns
    for i, region in enumerate(regions):
        p = plot_prediction_interval(mean, lower, upper, df_gt, region, p)
    ps.append(p)
    
    for d in [6, 13, 20]:
        if d >= len(df_gt):
            continue
        z = np.logical_and(df_gt.iloc[d] < upper.iloc[d], df_gt.iloc[d] > lower.iloc[d])
        acc = sum(z) / len(z)
        accs.append((date, d + 1, acc))

# plot accuracies        
accs = pd.DataFrame(accs, columns=['date', 'days', 'acc'])
p = plot_accuracy(accs, plevel, 'Confirmed Cases, Italy', {'2020-04-19'})
show(p, f'img/{region_short}_pi_accuracy.png')

../analysis.py:217: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mets["date"] = pd.to_datetime(mets["date"])


In [11]:
# plot prediciton intervals
grid = gridplot(ps, ncols=2, plot_width=430)
show(grid, f'img/{region_short}_pis.png')